$\large{\textbf{1.1 Formulation:}} \\ \text{Let the set of warehouses be W={Ahmedabad, Bengaluru, Chennai, Delhi, Kolkata, Lucknow, Mumbai, Hyderabad} and }\\ \text{the set of Markets be M={Hassan, Jodhpur, Trichy, Simla, Vijayawada, Dibrugarh, Raipur, Agartala}.} \\ \text{Let } c_{w,m} \text{ be the cost of transporting 1 load truck of EVs from warehouse 'w' to market 'm'.} \\ \text{Let } x_{w,m} \text{ be the number of load truck of EVs that moves from warehouse 'w' to market 'm'.} \\ \text{The set consisting of the available stocks for the diff warehouses S = {350,185,305,180,205,175,325,370} and } S_w \text{ is the available stock at the wth warehouse.} \\ \text{The set consisting of the demands at the various markets D = {205,300,425,270,375,225,185,110} and } D_m \text{ is the demand of the mth market.} \\ min \ \sum_{(w,m) \in (W,M)} c_{w,m} x_{w,m} \\ s.t. \ \sum_{m \in M} x_{w,m} \leq S_w \  ∀ \ w \in W \\ \  \sum_{w \in W} x_{w,m} = D_m \  ∀ \ m \in M \\ x_{w,m} \in \mathbf{Z}^+\ \ \forall \  (w,m) \in (W,M)$

In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.7 MB 4.2 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 


In [2]:
from pyomo.environ import *

In [3]:
import pandas as pd

In [4]:
data_csvfile = pd.read_csv('lab6_ex1.csv')

In [5]:
data_csvfile

,WH_MARKETS,Hassan,Jodhpur,Trichy,Simla,Vijayawada,Dibrugarh,Raipur,Agartala,Avail.Stock
0,Ahmedabad,827,817,1470,2272,1615,3143,1264,4465,350
1,Bengaluru,559,1832,751,3772,637,3773,1856,5459,185
2,Chennai,1109,2443,577,3865,798,3521,1984,5303,305
3,Delhi,2943,1471,4676,1337,2702,2741,2482,4148,180
4,Kolkata,3742,3089,2931,3199,2226,1739,1691,2128,205
5,Lucknow,2353,1160,3295,368,2369,2365,1293,3642,175
6,Mumbai,1356,1617,2045,2941,2334,4151,1937,4287,325
7,Hyderabad,195,1862,1331,2921,402,2825,563,4441,370
8,Demand,205,300,425,270,375,225,185,110,2095


In [6]:
data_csvfile.index

RangeIndex(start=0, stop=9, step=1)

In [7]:
data_csvfile.columns

Index(['WH_MARKETS', 'Hassan', 'Jodhpur', 'Trichy', 'Simla', 'Vijayawada',
       'Dibrugarh', 'Raipur', 'Agartala', 'Avail.Stock'],
      dtype='object')

In [8]:
model_ex1 = ConcreteModel()

In [10]:
#No of source locations
M = len(data_csvfile.index)-1

#No of destinations
N = len(data_csvfile.columns)-2

print('No of sources = {}'.format(M))
print('No of destinations = {}'.format(N))

No of sources = 8
No of destinations = 8


In [11]:
row_indices = range(M)
col_indices = range(N)

In [12]:
model_ex1.x = Var(row_indices, col_indices, domain = NonNegativeIntegers)

In [14]:
model_ex1.obj = Objective(expr = sum(data_csvfile.iloc[i,j+1]*model_ex1.x[i,j] for i in row_indices for j in col_indices), sense = minimize)

In [15]:
model_ex1.const = ConstraintList()

In [16]:
#Supply COnstraints:
for i in row_indices:
  model_ex1.const.add(expr = sum(model_ex1.x[i,j] for j in col_indices) <= data_csvfile.iloc[i,-1])


#Demand Constraits:
for j in col_indices:
  model_ex1.const.add(expr = sum(model_ex1.x[i,j] for i in row_indices) == data_csvfile.iloc[-1,j+1])

In [17]:
model_ex1.pprint()

4 Set Declarations
    const_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        No

In [18]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155569 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [19]:
opt_cbc = SolverFactory('cbc')

In [21]:
result = opt_cbc.solve(model_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [24]:
# display solution
print('\nObjective = ', model_ex1.obj())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    if(model_ex1.x[i,j].value != 0):
      print('Number of truck loads between ',data_csvfile['WH_MARKETS'][i],' and ', data_csvfile.columns[j+1],':', model_ex1.x[i,j].value)

print('\nConstraints')
model_ex1.const.display()


Objective =  2192870.0

Decision Variables
Number of truck loads between  Ahmedabad  and  Jodhpur : 300.0
Number of truck loads between  Ahmedabad  and  Dibrugarh : 45.0
Number of truck loads between  Ahmedabad  and  Raipur : 5.0
Number of truck loads between  Bengaluru  and  Trichy : 120.0
Number of truck loads between  Bengaluru  and  Vijayawada : 65.0
Number of truck loads between  Chennai  and  Trichy : 305.0
Number of truck loads between  Delhi  and  Simla : 95.0
Number of truck loads between  Delhi  and  Dibrugarh : 85.0
Number of truck loads between  Kolkata  and  Dibrugarh : 95.0
Number of truck loads between  Kolkata  and  Agartala : 110.0
Number of truck loads between  Lucknow  and  Simla : 175.0
Number of truck loads between  Mumbai  and  Hassan : 205.0
Number of truck loads between  Mumbai  and  Raipur : 120.0
Number of truck loads between  Hyderabad  and  Vijayawada : 310.0
Number of truck loads between  Hyderabad  and  Raipur : 60.0

Constraints
const : Size=16
    Key :

$\large{\textbf{Answer 1.7: }} \\ \text{Total Cost = Rs. 2192870} \\ \textbf{Number of truck loads of EVs:} \\ \text{Ahmedabad} → \text{Jodhpur  (300)} \\ \text{Ahmedabad} → \text{Dibrugarh  (45)} \\ \text{Ahmedabad} → \text{Raipur  (5)} \\ \text{Bengaluru} → \text{Trichy  (120)} \\ \text{Bengaluru} → \text{Vijayawada  (65)} \\ \text{Chennai} → \text{Trichy  (305)} \\ \text{Delhi} → \text{Simla  (95)} \\ \text{Delhi} → \text{Dibrugarh  (85)} \\ \text{Kolkata} → \text{Dibrugarh  (95)} \\ \text{Kolkata} → \text{Agartala  (110)} \\ \text{Lucknow} → \text{Simla  (175)} \\ \text{Mumbai} → \text{Hassan  (205)} \\ \text{Mumbai} → \text{Raipur  (120)} \\ \text{Hyderabad} → \text{Vijayawada  (310)} \\ \text{Hyderabad} → \text{Raipur  (60)}$

$\large{\textbf{Answer 1.8:}} \\ \text{To make sure, no transportation is done on Lucknow-Simla route, we assign a huge cost of transportation to the route.} \\ \text{So when the cost is to be minimized, by default no transportation will take place on that route.}$

In [25]:
data_csvfile.iloc[5,4] = 20000

In [26]:
data_csvfile

,WH_MARKETS,Hassan,Jodhpur,Trichy,Simla,Vijayawada,Dibrugarh,Raipur,Agartala,Avail.Stock
0,Ahmedabad,827,817,1470,2272,1615,3143,1264,4465,350
1,Bengaluru,559,1832,751,3772,637,3773,1856,5459,185
2,Chennai,1109,2443,577,3865,798,3521,1984,5303,305
3,Delhi,2943,1471,4676,1337,2702,2741,2482,4148,180
4,Kolkata,3742,3089,2931,3199,2226,1739,1691,2128,205
5,Lucknow,2353,1160,3295,20000,2369,2365,1293,3642,175
6,Mumbai,1356,1617,2045,2941,2334,4151,1937,4287,325
7,Hyderabad,195,1862,1331,2921,402,2825,563,4441,370
8,Demand,205,300,425,270,375,225,185,110,2095


In [27]:
model_ex1.new_obj = Objective(expr = sum(data_csvfile.iloc[i,j+1]*model_ex1.x[i,j] for i in row_indices for j in col_indices), sense = minimize)

In [30]:
model_ex1.obj.deactivate()

In [31]:
model_ex1.pprint()

4 Set Declarations
    const_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        No

In [32]:
result = opt_cbc.solve(model_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [33]:
# display solution
print('\nObjective = ', model_ex1.obj())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    if(model_ex1.x[i,j].value != 0):
      print('Number of truck loads between ',data_csvfile['WH_MARKETS'][i],' and ', data_csvfile.columns[j+1],':', model_ex1.x[i,j].value)

print('\nConstraints')
model_ex1.const.display()


Objective =  2380570.0

Decision Variables
Number of truck loads between  Ahmedabad  and  Jodhpur : 300.0
Number of truck loads between  Ahmedabad  and  Raipur : 50.0
Number of truck loads between  Bengaluru  and  Trichy : 120.0
Number of truck loads between  Bengaluru  and  Vijayawada : 65.0
Number of truck loads between  Chennai  and  Trichy : 305.0
Number of truck loads between  Delhi  and  Simla : 180.0
Number of truck loads between  Kolkata  and  Dibrugarh : 95.0
Number of truck loads between  Kolkata  and  Agartala : 110.0
Number of truck loads between  Lucknow  and  Dibrugarh : 130.0
Number of truck loads between  Lucknow  and  Raipur : 45.0
Number of truck loads between  Mumbai  and  Hassan : 205.0
Number of truck loads between  Mumbai  and  Simla : 90.0
Number of truck loads between  Mumbai  and  Raipur : 30.0
Number of truck loads between  Hyderabad  and  Vijayawada : 310.0
Number of truck loads between  Hyderabad  and  Raipur : 60.0

Constraints
const : Size=16
    Key : Lo

$\large{\textbf{Answer 1.9: }} \\ \text{Total Cost = Rs. 2380570} \\ \textbf{Number of truck loads of EVs:} \\ \text{Ahmedabad} → \text{Jodhpur  (300)} \\ \text{Ahmedabad} → \text{Raipur  (50)} \\ \text{Bengaluru} → \text{Trichy  (120)} \\ \text{Bengaluru} → \text{Vijayawada  (65)} \\ \text{Chennai} → \text{Trichy  (305)} \\ \text{Delhi} → \text{Simla  (180)} \\ \text{Kolkata} → \text{Dibrugarh  (95)} \\ \text{Kolkata} → \text{Agartala  (110)} \\ \text{Lucknow} → \text{Dibrugarh  (130)} \\ \text{Lucknow} → \text{Raipur  (45)} \\ \text{Mumbai} → \text{Hassan  (205)} \\ \text{Mumbai} → \text{Simla  (90)} \\ \text{Mumbai} → \text{Raipur  (30)} \\ \text{Hyderabad} → \text{Vijayawada  (310)} \\ \text{Hyderabad} → \text{Raipur  (60)} \\ \text{Here we can see that the total cost of operations increase when there was no transporation on the Lucknow-Simla route.}\\ \text{This is because in the ideal solution, minimum cost was attained when 175 trucks were moving from Lucknow to Simla,} \\ \text{but when we put a higher cost to the route, traffic was managed with different routes resulting in a comparitively higher cost.} $

$\large{\textbf{Answer 1.10:}}$

In [34]:
data_csvfile.iloc[4,6] = 20000
data_csvfile.iloc[3,7] = 20000

In [35]:
data_csvfile

,WH_MARKETS,Hassan,Jodhpur,Trichy,Simla,Vijayawada,Dibrugarh,Raipur,Agartala,Avail.Stock
0,Ahmedabad,827,817,1470,2272,1615,3143,1264,4465,350
1,Bengaluru,559,1832,751,3772,637,3773,1856,5459,185
2,Chennai,1109,2443,577,3865,798,3521,1984,5303,305
3,Delhi,2943,1471,4676,1337,2702,2741,20000,4148,180
4,Kolkata,3742,3089,2931,3199,2226,20000,1691,2128,205
5,Lucknow,2353,1160,3295,20000,2369,2365,1293,3642,175
6,Mumbai,1356,1617,2045,2941,2334,4151,1937,4287,325
7,Hyderabad,195,1862,1331,2921,402,2825,563,4441,370
8,Demand,205,300,425,270,375,225,185,110,2095


In [36]:
model_ex1.new_obj1 = Objective(expr = sum(data_csvfile.iloc[i,j+1]*model_ex1.x[i,j] for i in row_indices for j in col_indices), sense = minimize)

In [37]:
model_ex1.new_obj.deactivate()

In [38]:
result = opt_cbc.solve(model_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [39]:
# display solution
print('\nObjective = ', model_ex1.obj())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    if(model_ex1.x[i,j].value != 0):
      print('Number of truck loads between ',data_csvfile['WH_MARKETS'][i],' and ', data_csvfile.columns[j+1],':', model_ex1.x[i,j].value)

print('\nConstraints')
model_ex1.const.display()


Objective =  2518200.0

Decision Variables
Number of truck loads between  Ahmedabad  and  Jodhpur : 300.0
Number of truck loads between  Ahmedabad  and  Dibrugarh : 50.0
Number of truck loads between  Bengaluru  and  Trichy : 120.0
Number of truck loads between  Bengaluru  and  Vijayawada : 65.0
Number of truck loads between  Chennai  and  Trichy : 305.0
Number of truck loads between  Delhi  and  Simla : 180.0
Number of truck loads between  Kolkata  and  Raipur : 95.0
Number of truck loads between  Kolkata  and  Agartala : 110.0
Number of truck loads between  Lucknow  and  Dibrugarh : 175.0
Number of truck loads between  Mumbai  and  Hassan : 205.0
Number of truck loads between  Mumbai  and  Simla : 90.0
Number of truck loads between  Mumbai  and  Raipur : 30.0
Number of truck loads between  Hyderabad  and  Vijayawada : 310.0
Number of truck loads between  Hyderabad  and  Raipur : 60.0

Constraints
const : Size=16
    Key : Lower : Body  : Upper
      1 :  None : 350.0 : 350.0
      2

$\large{\textbf{Answer 1.11: }} \\ \text{Total Cost = Rs. 2518200} \\ \textbf{Number of truck loads of EVs:} \\ \text{Ahmedabad} → \text{Jodhpur  (300)} \\ \text{Ahmedabad} → \text{Dibrugarh  (50)} \\ \text{Bengaluru} → \text{Trichy  (120)} \\ \text{Bengaluru} → \text{Vijayawada  (65)} \\ \text{Chennai} → \text{Trichy  (305)} \\ \text{Delhi} → \text{Simla  (180)} \\ \text{Kolkata} → \text{Raipur  (95)} \\ \text{Kolkata} → \text{Agartala  (110)} \\ \text{Lucknow} → \text{Dibrugarh  (175)} \\ \text{Mumbai} → \text{Hassan  (205)} \\ \text{Mumbai} → \text{Simla  (90)} \\ \text{Mumbai} → \text{Raipur  (30)} \\ \text{Hyderabad} → \text{Vijayawada  (310)} \\ \text{Hyderabad} → \text{Raipur  (60)}\\ \text{Here also we can observe that the total cost of operations increase.}\\ \text{This is because in the ideal solution, while there was no transportation on the Delhi-Raipur route,} \\ \text{minimum cost was attained when 95 trucks were moving from Kolkata to Dibrugarh,} \\ \text{but when we put a higher cost to the route, traffic was managed with different routes resulting in a comparitively higher cost.}$